In [1]:
import pandas as pd
import numpy as np

# Table 1: Historical Usage Data
historical_usage_data = pd.DataFrame({
    'usage_id': range(1, 11),
    'product_group': ['c6i', 'c5', 'c6g', 'c6i', 'c5', 'c6g', 'c6i', 'c5', 'c6g', 'c6i'],
    'date': pd.date_range(start='2023-01-01', periods=10, freq='M'),
    'cpu_usage': np.random.randint(500, 1500, size=10),
    'memory_usage': np.random.randint(1000, 8000, size=10),
    'storage_usage': np.random.randint(10000, 50000, size=10)
})

# Save historical usage data to Parquet
historical_usage_data.to_parquet('historical_usage_data.parquet', index=False)

# Table 2: Forecasting Results
forecasting_results = pd.DataFrame({
    'forecast_id': range(1, 11),
    'product_group': ['c6i', 'c5', 'c6g', 'c6i', 'c5', 'c6g', 'c6i', 'c5', 'c6g', 'c6i'],
    'scenario_id': np.random.randint(1, 4, size=10),
    'forecast_date': pd.date_range(start='2024-01-01', periods=10, freq='M'),
    'cpu_forecast': np.random.randint(1000, 2000, size=10),
    'memory_forecast': np.random.randint(5000, 10000, size=10),
    'storage_forecast': np.random.randint(20000, 80000, size=10)
})

# Save forecasting results to Parquet
forecasting_results.to_parquet('forecasting_results.parquet', index=False)

# Table 3: Scenario Snapshots
scenario_snapshots = pd.DataFrame({
    'scenario_id': range(1, 4),
    'scenario_name': ['Base Case', 'High Demand', 'Low Demand'],
    'snapshot_date': pd.to_datetime(['2024-01-01', '2024-01-15', '2024-01-30']),
    'description': ['Normal growth forecast', 'Higher than expected growth', 'Lower than expected growth']
})

# Save scenario snapshots to Parquet
scenario_snapshots.to_parquet('scenario_snapshots.parquet', index=False)

# Table 4: Product Group CPU Mapping
product_group_cpu_mapping = pd.DataFrame({
    'product_group': ['c6i', 'c5', 'c6g'],
    'cpu_type': ['Intel Ice Lake', 'Intel Cascade Lake', 'Graviton2'],
    'generation': ['Ice Lake', 'Cascade Lake', 'Graviton2']
})

# Save product group CPU mapping to Parquet
product_group_cpu_mapping.to_parquet('product_group_cpu_mapping.parquet', index=False)

# Table 5: Product Group Details
product_group_details = pd.DataFrame({
    'product_group': ['c6i', 'c5', 'c6g'],
    'vcpus': [64, 48, 64],
    'memory_gb': [128, 96, 128],
    'storage_gb': [2000, 1500, 2000],
    'network_performance': ['Up to 25 Gbps', 'Up to 10 Gbps', 'Up to 25 Gbps']
})

# Save product group details to Parquet
product_group_details.to_parquet('product_group_details.parquet', index=False)

print("All tables have been saved as Parquet files.")


All tables have been saved as Parquet files.


In [5]:
import polars as pl


In [6]:
pl.read_parquet('product_group_cpu_mapping.parquet')

product_group,cpu_type,generation
str,str,str
"""c6i""","""Intel Ice Lake""","""Ice Lake"""
"""c5""","""Intel Cascade Lake""","""Cascade Lake"""
"""c6g""","""Graviton2""","""Graviton2"""


In [22]:
import polars as pl
import os

class PH:
    def __init__(self, directory="."):
        self.dataframes = {}
        self.load_parquet_files(directory)

    def read_parquet(self, file_path, table_name):
        """
        Reads a Parquet file into a Polars DataFrame and stores it in the handler with the given table_name.
        """
        try:
            df = pl.read_parquet(file_path)
            self.dataframes[table_name] = df
            print(f"Successfully read {file_path} into table '{table_name}'")
        except Exception as e:
            print(f"Failed to read {file_path}: {e}")
            
    def load_parquet_files(self, directory):
        """
        Load all Parquet files in the specified directory into Polars DataFrames.
        
        Args:
        directory (str): The directory where the Parquet files are stored.
        """
        for filename in os.listdir(directory):
            if filename.endswith(".parquet"):
                table_name = os.path.splitext(filename)[0]
                file_path = os.path.join(directory, filename)
                self.dataframes[table_name] = pl.read_parquet(file_path)
                print(f"Loaded {file_path} as table '{table_name}'")
    
    
    def query(self, sql_query): #self, table_name, sql_query
        """
        Execute an SQL query using Polars SQL context.
        
        Args:
        sql_query (str): The SQL query string.
        
        Returns:
        Polars DataFrame: The result of the SQL query.
        """
        context = pl.SQLContext(self.dataframes)
        try:
            result = context.execute(sql_query).collect()
            return result
        except Exception as e:
            print(f"Query execution error: {e}")
            return None
#         """
#         Queries a stored DataFrame using Polars SQL and returns the result.
#         """
#         if table_name in self.dataframes:
#             try:
#                 context = pl.SQLContext(self.dataframes)
#                 result = context.execute(sql_query).collect()
#                 return result
#             except Exception as e:
#                 print(f"Failed to execute query: {e}")
#                 return None
#         else:
#             print(f"Table '{table_name}' not found.")
#             return None
    
    def get_table(self, table_name):
        """
        Returns the Polars DataFrame for the given table_name.
        """
        if table_name in self.dataframes:
            return self.dataframes[table_name]
        else:
            print(f"Table '{table_name}' not found.")
            return None

# Example usage:
# Initialize the SQL handler
sql_handler = PH()

# Example query
# query = "SELECT AVG(cpu_usage) FROM historical_usage_data WHERE product_group = 'c6i' AND date = '2023-01-31';"
query = 'SELECT * FROM scenario_snapshots'
result = sql_handler.query(query)

if result is not None:
    print(result)

# # Create the handler
# handler = PolarHandler()

# # Read Parquet files into tables
# handler.read_parquet('historical_usage_data.parquet', 'historical_usage')
# handler.read_parquet('forecasting_results.parquet', 'forecasting_results')
# handler.read_parquet('scenario_snapshots.parquet', 'scenario_snapshots')
# handler.read_parquet('product_group_cpu_mapping.parquet', 'product_group_cpu_mapping')
# handler.read_parquet('product_group_details.parquet', 'product_group_details')

# # Direct access example
# historical_usage_df = handler.get_table('historical_usage')
# print(historical_usage_df)


Loaded ./product_group_cpu_mapping.parquet as table 'product_group_cpu_mapping'
Loaded ./historical_usage_data.parquet as table 'historical_usage_data'
Loaded ./product_group_details.parquet as table 'product_group_details'
Loaded ./forecasting_results.parquet as table 'forecasting_results'
Loaded ./scenario_snapshots.parquet as table 'scenario_snapshots'
shape: (3, 4)
┌─────────────┬───────────────┬─────────────────────┬─────────────────────────────┐
│ scenario_id ┆ scenario_name ┆ snapshot_date       ┆ description                 │
│ ---         ┆ ---           ┆ ---                 ┆ ---                         │
│ i64         ┆ str           ┆ datetime[ns]        ┆ str                         │
╞═════════════╪═══════════════╪═════════════════════╪═════════════════════════════╡
│ 1           ┆ Base Case     ┆ 2024-01-01 00:00:00 ┆ Normal growth forecast      │
│ 2           ┆ High Demand   ┆ 2024-01-15 00:00:00 ┆ Higher than expected growth │
│ 3           ┆ Low Demand    ┆ 2024-01-

In [26]:
type(sql_handler.get_table('scenario_snapshots'))

polars.dataframe.frame.DataFrame

In [12]:
# Query example
query_result = handler.query('scenario_snapshots', 
                             "SELECT * FROM scenario_snapshots")
print(query_result)


shape: (3, 4)
┌─────────────┬───────────────┬─────────────────────┬─────────────────────────────┐
│ scenario_id ┆ scenario_name ┆ snapshot_date       ┆ description                 │
│ ---         ┆ ---           ┆ ---                 ┆ ---                         │
│ i64         ┆ str           ┆ datetime[ns]        ┆ str                         │
╞═════════════╪═══════════════╪═════════════════════╪═════════════════════════════╡
│ 1           ┆ Base Case     ┆ 2024-01-01 00:00:00 ┆ Normal growth forecast      │
│ 2           ┆ High Demand   ┆ 2024-01-15 00:00:00 ┆ Higher than expected growth │
│ 3           ┆ Low Demand    ┆ 2024-01-30 00:00:00 ┆ Lower than expected growth  │
└─────────────┴───────────────┴─────────────────────┴─────────────────────────────┘


In [13]:
TABLE_DETAILS = {
    "historical_usage": "This table stores historical usage data for different product groups, including CPU, memory, and storage usage.",
    "forecasting_results": "This table contains forecasting results based on various scenarios, including future CPU, memory, and storage needs.",
    "scenario_snapshots": "This table holds snapshots of different scenarios used in forecasting, with descriptions and snapshot dates.",
    "product_group_cpu_mapping": "This table maps product groups to their respective CPU types and generations.",
    "product_group_details": "This table provides detailed information about each product group, including vCPUs, memory, storage, and network performance.",
}


SQL_TEMPLATE_STR = """Given an input question, first create a syntactically correct SQL query to run, then look at the results of the query and return the answer.
    You can order the results by a relevant column to return the most insightful data in the database.\n\n
    Never query for all the columns from a specific table, only ask for a few relevant columns given the question.\n\n
    Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist.
    Qualify column names with the table name when needed.

    If a column name contains a space, always wrap the column name in double quotes.

    You are required to use the following format, each taking one line:\n\nQuestion: Question here\nSQLQuery: SQL Query to run\n
    SQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nOnly use tables listed below.\n{schema}\n\n
    Do not under any circumstance use SELECT * in your query.
    If the user is asking about hardware usage—interpret it as querying the historical usage data.

    Here are some useful examples:
    {few_shot_examples}

    Question: {query_str}\nSQLQuery: """

RESPONSE_TEMPLATE_STR = """If the <SQL Response> below contains data, then given an input question, synthesize a response from the query results.
    If the <SQL Response> is empty, then you should not synthesize a response and instead respond that no data was found for the question.\n

    \nQuery: {query_str}\nSQL: {sql_query}\n<SQL Response>: {context_str}\n</SQL Response>\n

    Do not make any mention of queries or databases in your response, instead you can say 'according to the latest data' .\n\n
    Please make sure to mention any additional details from the context supporting your response.
    If the final answer contains <dollar_sign>$</dollar_sign>, ADD '\' ahead of each <dollar_sign>$</dollar_sign>.

    Response: """